In [1]:
import requests
import json
import geopandas as gpd
import pandas as pd

In [2]:
# pulling in permit data from permit acquisition shapefile
permitdata=gpd.read_file('/Users/alliepadgett/Documents/GitHub/Housing_Modeling/Data/permit_gdf.geojson')

In [3]:
# checking everything!
permitdata.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 27742 entries, 0 to 27741
Data columns (total 58 columns):
 #   Column                                   Non-Null Count  Dtype   
---  ------                                   --------------  -----   
 0   assessor_parcel                          27709 non-null  object  
 1   zip_code                                 27742 non-null  object  
 2   location_1_address                       27742 non-null  object  
 3   work_description                         27742 non-null  object  
 4   applicant_address_3                      21295 non-null  object  
 5   floor_area_l_a_zoning_code_definition    26987 non-null  object  
 6   address_fraction_end                     2317 non-null   object  
 7   project_number                           135 non-null    object  
 8   suffix_direction                         41 non-null     object  
 9   of_stories                               27735 non-null  object  
 10  address_start             

In [4]:
# pulling in clipped LA shapefile with census data
lacensus=gpd.read_file('/Users/alliepadgett/Documents/GitHub/Housing_Modeling/Data/lacounty_clipped.geojson')


In [5]:
# setting projections and spatial joining census data to permit data to get tracts for each permit
# lacensus = clipped shapefile w tracts & census variables
# permitdata = permit info
# permitcensus = permit info w/ tracts assigned
permitdata=permitdata.set_crs('EPSG:4326')
lacensus=lacensus.to_crs('EPSG:4326')
permitcensus = gpd.sjoin(permitdata, lacensus, how="left", predicate='intersects')
permitcensus.head()

,assessor_parcel,zip_code,location_1_address,work_description,applicant_address_3,floor_area_l_a_zoning_code_definition,address_fraction_end,project_number,suffix_direction,of_stories,...,EducMasters,EducProf,EducDoct,PctNHW,PctBlackAA,PctAsian,pctPOC,PctBachelor,PctMasters,PctDoctorate
0,029,90038,,New 3-Story Duplex w/Attached 2-Car Garage Eac...,None,2328,None,None,None,3,...,312.0,142.0,27.0,84.038737,0.645624,6.420373,15.961263,50.249660,20.608261,1.225601
1,013,90007,,"BUILDING ""B"" - NEW (3) STORY DUPLEX BUILDING A...","LAWNDALE, CA",4963,1/2,None,None,3,...,205.0,18.0,30.0,26.166877,18.489818,18.760137,73.833123,15.025413,7.083863,0.952986
2,033,91326,,TWO STORY SFD WITH ATTACHED 3 CAR GARAGE _STAN...,PORTER RANCH,4957,None,None,None,2,...,987.0,474.0,119.0,39.121143,7.853613,48.176840,60.878857,33.094340,27.566038,2.245283
3,008,90066,,New SFD with attached garage and recroom.,None,5994,None,None,None,2,...,446.0,287.0,69.0,74.333770,0.917431,5.635649,25.666230,36.698622,21.959257,2.067106
4,010,90018,,NEW TWO STORY DUPLEX WITH OPEN PARKING AT REAR.,None,2444,1/2,None,None,2,...,6.0,59.0,0.0,31.139420,27.270583,5.425808,68.860580,21.167883,2.497119,0.000000


In [6]:
# make sure parcels are unique before using in a groupby
pd.Series('permitcensus.assessor_parcel').is_unique

True

In [7]:
# groupby to get number of permits per tract
permit_counts=permitcensus.groupby(['GEOID']).count()['assessor_parcel']
permit_counts

GEOID
06037101110    48
06037101122    25
06037101210    18
06037101220    15
06037101300    26
               ..
06037980021     1
06037980022     1
06037980024     7
06037980028    69
06037980031    26
Name: assessor_parcel, Length: 970, dtype: int64

In [8]:
# join permit counts back to census shapefile
permitcensus.set_index('GEOID', inplace=True)

permit_counts.name = 'permit_count'
censuspermitcounts = lacensus.join(permit_counts, on = 'GEOID', how='left')
censuspermitcounts.head()

,GEOID,TotalPop,WhitePop,BlackorAA,AmInd_AKnative,Asian,Hawaiian_PI,Other,EducTotal,Median HHI,...,EducDoct,PctNHW,PctBlackAA,PctAsian,pctPOC,PctBachelor,PctMasters,PctDoctorate,geometry,permit_count
0,06037274202,4284.0,3340.0,88.0,0.0,352.0,0.0,382.0,3786.0,128766.0,...,247.0,77.964519,2.054155,8.216620,22.035481,34.970946,27.284733,6.524036,"POLYGON ((-118.46683 33.97905, -118.46654 33.9...",8.0
1,06037273902,4221.0,3707.0,76.0,13.0,162.0,0.0,36.0,3601.0,144924.0,...,99.0,87.822791,1.800521,3.837953,12.177209,43.043599,23.965565,2.749236,"POLYGON ((-118.47136 33.98425, -118.47106 33.9...",48.0
2,06037273502,2861.0,2384.0,207.0,18.0,58.0,0.0,45.0,2487.0,90161.0,...,58.0,83.327508,7.235232,2.027263,16.672492,44.873341,21.431444,2.332127,"POLYGON ((-118.47671 33.98745, -118.47504 33.9...",28.0
3,06037273402,2641.0,2432.0,157.0,0.0,5.0,0.0,12.0,2386.0,72258.0,...,98.0,92.086331,5.944718,0.189322,7.913669,40.067058,23.470243,4.107293,"POLYGON ((-118.48334 33.99551, -118.48217 33.9...",21.0
4,06037273300,2915.0,2085.0,446.0,0.0,70.0,0.0,223.0,2297.0,71823.0,...,46.0,71.526587,15.300172,2.401372,28.473413,47.757945,13.757074,2.002612,"POLYGON ((-118.47838 33.99813, -118.47816 33.9...",54.0


In [9]:
censuspermitcounts.info()


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 1149 entries, 0 to 1148
Data columns (total 27 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   GEOID           1149 non-null   object  
 1   TotalPop        1149 non-null   float64 
 2   WhitePop        1149 non-null   float64 
 3   BlackorAA       1149 non-null   float64 
 4   AmInd_AKnative  1149 non-null   float64 
 5   Asian           1149 non-null   float64 
 6   Hawaiian_PI     1149 non-null   float64 
 7   Other           1149 non-null   float64 
 8   EducTotal       1149 non-null   float64 
 9   Median HHI      1133 non-null   float64 
 10  NAME            1149 non-null   object  
 11  state           1149 non-null   object  
 12  county          1149 non-null   object  
 13  tract           1149 non-null   object  
 14  EducBachelor    1149 non-null   float64 
 15  EducMasters     1149 non-null   float64 
 16  EducProf        1149 non-null   float64 
 17  EducDo

In [11]:
# renaming for clarity
CensusPlusPermits = censuspermitcounts

In [12]:
# saving down the file, to bring into other notebooks!

CensusPlusPermits.to_file("Data/CensusPlusPermits.geojson", driver='GeoJSON')


/Users/alliepadgett/opt/anaconda3/envs/uds/lib/python3.8/site-packages/geopandas/io/file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
